# Build Metric Groups

This worksheet builds an `AdditiveAbelianGroup` and a homogeneous nondegenerate quadratic form and its associated nondegenerate bilinear form given a list of tuples of the form $(p,r,d)$ where $p$ is a prime, $r$ is a positive integer, and $d$ is an integer between 1 and 2 if $p$ is odd, and $d$ is between 1 and 6 if $p=2$.  The number $d$ determines the type of quadratic form as defined by Miranda.

To create the quadratic form, it is necessesary to create the smallest possible cyclotomic field that will be needed for the quadratic form, and will also contain 4th or 8th roots for Gauss sums and the square root of the order of the group.

In [2]:
import itertools

In [3]:
def cyclotomic_n(pr_orders):
    n = prod(pr_orders)
    n1 = lcm(pr_orders)
    #n1 corresponds to denominator of the quadratic form
    if n1 % 2 == 0:
        n1 = n1*2
    #if n1 is odd, gauss sums will exist in QQ(i)
    else:
        n1 = n1*4
    #We need a cyclotomic field that contains sqrt(n)
    n2 = n.squarefree_part()
    if n2 % 2 == 0:
        n2 = n2 * 4
    n3 = lcm(n1,n2)
    return n3

In [4]:
def build_metric_group(prd_tuples):
    orders = [p^r for (p,r,d) in prd_tuples for i in range(int(d/5)+1)]
    n = cyclotomic_n(orders)
    G = AdditiveAbelianGroup(orders)
    Z = CyclotomicField(n)
    z = Z.gen()
    Q = []
    for tup in prd_tuples:
        p,r,d = tup
        if p == 2 and d == 1:
            Q.append(matrix([[n*p^(-r-1)]]))
        elif p == 2 and d == 2:
            Q.append(matrix([[-n*p^(-r-1)]]))
        elif p == 2 and d == 3:
            Q.append(matrix([[5*n*p^(-r-1)]]))
        elif p == 2 and d == 4:
            Q.append(matrix([[-5*n*p^(-r-1)]]))
        elif p == 2 and d == 5:
            Q.append(n*p^(-r-1)*matrix([[0,1],[1,0]]))
        elif p == 2 and d == 6:
            Q.append(n*p^(-r-1)*matrix([[2,1],[1,2]]))
        elif d == 1:
            v = (p^r + 1)/2 #v is 2^(-1) in Z/p^rZ
            Q.append(matrix([[ZZ(v*n*p^(-r))]]))
        elif d == 2:
            v = (p^r + 1)/2 #v is 2^(-1) in Z/p^rZ
            u = ZZ(Zmod(p).multiplicative_generator())
            Q.append(matrix([[ZZ(u*v*n*p^(-r))]]))
        else:
            print("Build error: ", p, r, d)
    return [G,block_diagonal_matrix(Q, subdivide=False),z]

The functions `q` and `b` are multiplicative and the functions `q2` and `b2` are additive.

In [5]:
def q(g,Q,z):
    x = vector(g.lift())
    return z^(x*Q*x)

def b(g,h,Q,z):
    return q(g+h,Q,z)/q(g,Q,z)/q(h,Q,z)

def q2(g,Q,z):
    n = z.multiplicative_order()
    x = vector(g.lift())
    return 1/n*(x*Q*x % n)

def b2(g,h,Q,z):
    return q2(g+h,Q,z)-q2(g,Q,z)-q2(h,Q,z)

In [13]:
G, Q, z = build_metric_group([(2,2,1),(3,1,1)])
show(G,Q,z)

Additive abelian group isomorphic to Z/4 + Z/3 [ 3  0]
[ 0 16] zeta24

In [14]:
show(matrix([[b2(g,h,Q,z) for g in G.list()] for h in G.list()]))

[     0      0      0      0      0      0      0      0      0      0      0      0]
[     0 -17/12   -5/6   -1/4   -2/3 -13/12   -1/2 -11/12   -4/3   -3/4   -1/6 -19/12]
[     0   -5/6    1/3    1/2   -1/3   -1/6      0   -5/6   -2/3    1/2   -1/3   -1/6]
[     0   -1/4    1/2    1/4      0   -1/4   -1/2   -3/4      0   -1/4    1/2   -3/4]
[     0   -2/3   -1/3      0   -2/3   -4/3     -1   -2/3   -4/3      0   -2/3   -4/3]
[     0 -13/12   -1/6   -1/4   -4/3 -17/12   -1/2 -19/12   -2/3   -3/4   -5/6 -11/12]
[     0   -1/2      0   -1/2     -1   -1/2     -1   -1/2     -1   -1/2      0   -1/2]
[     0 -11/12   -5/6   -3/4   -2/3 -19/12   -1/2 -17/12   -4/3   -1/4   -1/6 -13/12]
[     0   -4/3   -2/3      0   -4/3   -2/3     -1   -4/3   -2/3      0   -1/3   -2/3]
[     0   -3/4    1/2   -1/4      0   -3/4   -1/2   -1/4      0    1/4    1/2   -1/4]
[     0   -1/6   -1/3    1/2   -2/3   -5/6      0   -1/6   -1/3    1/2    1/3   -5/6]
[     0 -19/12   -1/6   -3/4   -4/3 -11/12   -1/2 -13/12   -2/3   -1/4   -5/6 -17/12]

In [15]:
def signature(G,Q,z):
    v = ZZ.valuation(2)
    gens = G.gens()
    #determine the power of 2 of tuple
    ords = [v(g.order()) for g in G.gens()]
    r = max(ords)
    l = len(ords)
    sqrt2 = z^ceil(2^(r-2)) + z^floor(-2^(r-2)) #ceil and floor help deal with the r=1 case
    sigma = []
    for k in range(r+1):
        #Note to self: no need to build H.  We just need it's size.
        direct_p = [[i*gens[j] for i in range(2^(max(ords[j]-k,0)))] for j in range(l)]
        H = [sum(list(a)) for a in itertools.product(*direct_p)]
        sqrt_H = sqrt2^(v(len(H)))
        sigma += [sqrt_H/G.order()*sum([q(g,Q,z)^(2^k) for g in G.list()])]
    return sigma

In [16]:
print(signature(G,Q,z))

[-2/3*zeta24^7 - 1/2*zeta24^5 + 1/3*zeta24^3 + 1/2*zeta24, 1/3*zeta24^7 + 1/2*zeta24^5 - 1/6*zeta24^3 - 1/2*zeta24, 0]


In [20]:
G, Q, z = build_metric_group([(2,2,1),(3,1,1)])
show(G,Q,z)

Additive abelian group isomorphic to Z/4 + Z/3 [ 3  0]
[ 0 16] zeta24

In [21]:
n = G.order()
primes = [p for (p,r) in factor(n)]
v_p = ZZ.valuation(p)
gens = G.gens()
ords = [g.order() for g in G.gens()]
ords_v = [v_p(o) for o in ords]
#determine the power of 2 of tuple
r = max(ords_v)
l = len(ords_v)
sqrt_p = G.sq
print(sqrt2, sqrt2^2)

-zeta24^7 + zeta24^3 + zeta24 -zeta24^6 + 2*zeta24^2 + 2


In [22]:
ords

[2, 0]

In [24]:
k = 0
direct_p = [[i*gens[j] for i in range(2^(max(ords[j]-k,0)))] for j in range(l)]
direct_p

[[(0, 0), (1, 0), (2, 0), (3, 0)], [(0, 0)]]

In [25]:
k = 1
direct_p = [[i*gens[j] for i in range(2^(max(ords[j]-k,0)))] for j in range(l)]
direct_p

[[(0, 0), (1, 0)], [(0, 0)]]

In [26]:
k = 2
direct_p = [[i*gens[j] for i in range(2^(max(ords[j]-k,0)))] for j in range(l)]
direct_p

[[(0, 0)], [(0, 0)]]

In [29]:
n = 12
[(p,r) for (p,r) in factor(n)]

[(2, 2), (3, 1)]